# DBN computing package boosted by GPU required:

In [ ]:
!pip install git+git://github.com/albertbup/deep-belief-network.git@master_gpu

In [ ]:
###########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
from dbn.tensorflow import SupervisedDBNRegression
# %matplotlib
###########assign memory##########
###########delete this part if your tensorflow was based on CPU##########
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 
###########fix random seed for reproducability##########
seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv('0721platinum.csv',encoding="gbk")
raw_data=fdata.loc[:,[                     
                      'Pt at% in metal element',#0
                      'Co at% in metal element',#1
                      'total metal mass ratio wt%',#2
                      'C wt%',#3
                      'Particle diameter （nm）',#4
                      'support BET surface area(m2/g)' ,#5
                      'Reduction Temperature',#6
                      'Reduction Time/min',#7
                      'Annealing Temperature',#8
                      'ECSA m2/g',#9
                      'Mass Activity mA mg-1',#10
                      'I/C Ratio(ionomer/catalyst)',#11
                      'Area cm2',#12
                      'Cathodic Loading Amount mgPt cm-2',#13
                      'Anodic Platinum Loading Amount mgPt cm-2',#14
                      'Anodic catalyst type x wt% Pt/C',#15
                      'membrane thickness',#16
                      'Hot Press Temperature',#17
                      'Hot Press Time min',#18
                      'Hot Press Pressure Mpa',#19
                      'Humidity %',#20
                      'GDE for 1',#21
                      'celltemp',#22
                      'Flowing rate of H2 ml min-1',#23
                      'flowing rate of cathode gas(O2/air)',#24
                      'Back Pressure Mpa',#25
                      'Cathode gas oxygen ratio',#26
                      'Open Circuit Voltage V',#27
                      'Voltage 100mA cm0',
                      'Voltage 200mA cm0',
                      'Voltage 300mA cm0',
                      'Voltage 400mA cm0',
                      'Voltage 500mA cm0',
                      'Voltage 600mA cm0',
                      'Voltage 700mA cm0',
                      'Voltage 800mA cm0',
                      'Voltage 900mA cm0',
                      'Voltage 1000mA cm0',
                      'Voltage 1100mA cm0',
                      'Voltage 1200mA cm0',
                      'Voltage 1300mA cm0',
                      'Voltage 1400mA cm0',
                      'Voltage 1500mA cm0',
                      'Voltage 1600mA cm0',
                      'Voltage 1700mA cm0',
                      'Voltage 1800mA cm0',
                      'Voltage 1900mA cm0',
                      'Voltage 2000mA cm0',
                      'Voltage 2100mA cm0',
                      'Voltage 2200mA cm0',
                      'Voltage 2300mA cm0',
                      'Voltage 2400mA cm0',
                      'Voltage 2500mA cm0',
                      'Voltage 2600mA cm0',
                      'Voltage 2700mA cm0',
                      'Voltage 2800mA cm0',
                      'Voltage 2900mA cm0',
                      'Voltage 3000mA cm0'#57
                        ]]
###########handling missing values##########
median_raw_data=raw_data.median()
dict_median_raw_data=median_raw_data.to_dict()
data=raw_data.fillna(dict_median_raw_data)
###########data standardization##########
standardized_data = (data-np.mean(data,axis=0))/np.std(data,axis=0)
###########train test splitting##########
#input allocation
median=standardized_data.median()
raw_median=data.median()
user_input1=raw_median[0:27]
standardized_user_input=(user_input1-np.mean(data,axis=0)[0:27])/np.std(data,axis=0)[0:27]
###########defining a wrapper function for later call from each machine learning algorithms##########
raw_param=standardized_data.iloc[:,0:27]
raw_power=standardized_data.iloc[:,27:]
raw_param_global=data.iloc[:,0:27]
raw_power_global=data.iloc[:,27:]
X_train=raw_param.drop(index=[235,381,456,619,582,583])
y_train=raw_power.drop(index=[235,381,456,619,582,583])
X_train=X_train.values.astype(np.float32)
y_train=y_train.values.astype(np.float32)
X_test_456=standardized_data.iloc[456,0:27]
X_test_235=standardized_data.iloc[235,0:27]
X_test_619=standardized_data.iloc[619,0:27]
X_test_381=standardized_data.iloc[381,0:27]
X_test_582=standardized_data.iloc[582,0:27]
X_test_583=standardized_data.iloc[583,0:27]

print('start')
for neuron_num in range(1600,2000,50):
    for lr_rbm in [0.01,0.02,0.05]:
        for lr in [0.01,0.02,0.05]:
            for n_epoch in range(3,9):
                for n_iter_back in range(640,1280,64):
                    for bs in range(32,80,16):
                        regressor = SupervisedDBNRegression(hidden_layers_structure=[neuron_num,neuron_num],
                                                                                    learning_rate_rbm=lr_rbm,
                                                                                        learning_rate=lr,
                                                                                        n_epochs_rbm=n_epoch,
                                                                                        n_iter_backprop=n_iter_back,
                                                                                        batch_size=bs,
                                                                                        activation_function='relu')
                        regressor.fit(X_train,y_train)
                        best_model=regressor
                        ###########get the prediction##########

                        data_test_input_235=X_test_235
                        data_test_input_235=pd.DataFrame(data_test_input_235)
                        data_test_input_235=data_test_input_235.T
                        data_test_param_235=data_test_input_235.values.astype(np.float32)
                        predict_ann_235_each= best_model.predict(data_test_param_235)

                        data_test_input_456=X_test_456
                        data_test_input_456=pd.DataFrame(data_test_input_456)
                        data_test_input_456=data_test_input_456.T
                        data_test_param_456=data_test_input_456.values.astype(np.float32)
                        predict_ann_456_each= best_model.predict(data_test_param_456)

                        data_test_input_381=X_test_381
                        data_test_input_381=pd.DataFrame(data_test_input_381)
                        data_test_input_381=data_test_input_381.T
                        data_test_param_381=data_test_input_381.values.astype(np.float32)
                        predict_ann_381_each= best_model.predict(data_test_param_381)


                        data_test_input_619=X_test_619
                        data_test_input_619=pd.DataFrame(data_test_input_619)
                        data_test_input_619=data_test_input_619.T
                        data_test_param_619=data_test_input_619.values.astype(np.float32)
                        predict_ann_619_each= best_model.predict(data_test_param_619)

                        data_test_input_582=X_test_582
                        data_test_input_582=pd.DataFrame(data_test_input_582)
                        data_test_input_582=data_test_input_582.T
                        data_test_param_582=data_test_input_582.values.astype(np.float32)
                        predict_ann_582_each= best_model.predict(data_test_param_582)

                        data_test_input_583=X_test_583
                        data_test_input_583=pd.DataFrame(data_test_input_583)
                        data_test_input_583=data_test_input_583.T
                        data_test_param_583=data_test_input_583.values.astype(np.float32)
                        predict_ann_583_each= best_model.predict(data_test_param_583)

                        ###########transfer the prediction to voltage values##########619
                        voltage_result_list_619=np.std(raw_power_global,axis=0).T.values*predict_ann_619_each+np.mean(raw_power_global,axis=0).T.values
                        ###########computing output power##########
                        x_current=np.arange(0,3100,100)
                        predict_power_619=x_current*voltage_result_list_619
                        predict_power_619=predict_power_619.T
                        ###########real values in literature##########
                        curve_real_619=data.iloc[619,27:]
                        predict_power_real_619=x_current*data.iloc[619,27:].values
                        ###########computing R2##########
                        real_current_series_619=pd.Series(data.iloc[619,27:48].values)
                        voltage_result_series_619=pd.Series(voltage_result_list_619[0,0:21])
                        corr_ann_619 = round(voltage_result_series_619.corr(real_current_series_619), 4)
                        rmse_val_619= rmse(data.iloc[619,27:48].values,voltage_result_list_619[0,0:21])  


                        ###########transfer the prediction to voltage values##########235
                        voltage_result_list_235=np.std(raw_power_global,axis=0).T.values*predict_ann_235_each+np.mean(raw_power_global,axis=0).T.values
                        ###########computing output power##########
                        x_current=np.arange(0,3100,100)
                        predict_power_235=x_current*voltage_result_list_235
                        predict_power_235=predict_power_235.T
                        ###########real values in literature##########
                        curve_real_235=data.iloc[235,27:]
                        predict_power_real_235=x_current*data.iloc[235,27:].values
                        ###########computing R2##########
                        real_current_series_235=pd.Series(data.iloc[235,27:48].values)
                        voltage_result_series_235=pd.Series(voltage_result_list_235[0,0:21])
                        corr_ann_235 = round(voltage_result_series_235.corr(real_current_series_235), 4)
                        rmse_val_235= rmse(data.iloc[235,27:48].values,voltage_result_list_235[0,0:21]) 

                        ###########transfer the prediction to voltage values##########381
                        voltage_result_list_381=np.std(raw_power_global,axis=0).T.values*predict_ann_381_each+np.mean(raw_power_global,axis=0).T.values
                        ###########computing output power##########
                        x_current=np.arange(0,3100,100)
                        predict_power_381=x_current*voltage_result_list_381
                        predict_power_381=predict_power_381.T
                        ###########real values in literature##########
                        curve_real_381=data.iloc[381,27:]
                        predict_power_real_381=x_current*data.iloc[381,27:].values
                        ###########computing R2##########
                        real_current_series_381=pd.Series(data.iloc[381,27:48].values)
                        voltage_result_series_381=pd.Series(voltage_result_list_381[0,0:21])
                        corr_ann_381 = round(voltage_result_series_381.corr(real_current_series_381), 4)
                        rmse_val_381= rmse(data.iloc[381,27:48].values,voltage_result_list_381[0,0:21])


                        ###########transfer the prediction to voltage values##########456
                        voltage_result_list_456=np.std(raw_power_global,axis=0).T.values*predict_ann_456_each+np.mean(raw_power_global,axis=0).T.values
                        ###########computing output power##########
                        x_current=np.arange(0,3100,100)
                        predict_power_456=x_current*voltage_result_list_456
                        predict_power_456=predict_power_456.T
                        ###########real values in literature##########
                        curve_real_456=data.iloc[456,27:]
                        predict_power_real_456=x_current*data.iloc[456,27:].values
                        ###########computing R2##########
                        real_current_series_456=pd.Series(data.iloc[456,27:44].values)
                        voltage_result_series_456=pd.Series(voltage_result_list_456[0,0:17])
                        corr_ann_456 = round(voltage_result_series_456.corr(real_current_series_456), 4)
                        rmse_val_456= rmse(data.iloc[456,27:44].values,voltage_result_list_456[0,0:17])  

                        ###########transfer the prediction to voltage values##########582
                        voltage_result_list_582=np.std(raw_power_global,axis=0).T.values*predict_ann_582_each+np.mean(raw_power_global,axis=0).T.values
                        ###########computing output power##########
                        x_current=np.arange(0,3100,100)
                        predict_power_582=x_current*voltage_result_list_582
                        predict_power_582=predict_power_582.T
                        ###########real values in literature##########
                        curve_real_582=data.iloc[582,27:]
                        predict_power_real_582=x_current*data.iloc[582,27:].values
                        ###########computing R2##########
                        real_current_series_582=pd.Series(data.iloc[582,27:53].values)
                        voltage_result_series_582=pd.Series(voltage_result_list_582[0,0:26])
                        corr_ann_582 = round(voltage_result_series_582.corr(real_current_series_582), 4)
                        rmse_val_582= rmse(data.iloc[582,27:53].values,voltage_result_list_582[0,0:26])  

                        ###########transfer the prediction to voltage values##########583
                        voltage_result_list_583=np.std(raw_power_global,axis=0).T.values*predict_ann_583_each+np.mean(raw_power_global,axis=0).T.values
                        ###########computing output power##########
                        x_current=np.arange(0,3100,100)
                        predict_power_583=x_current*voltage_result_list_583
                        predict_power_583=predict_power_583.T
                        ###########real values in literature##########
                        curve_real_583=data.iloc[583,27:]
                        predict_power_real_583=x_current*data.iloc[583,27:].values
                        ###########computing R2##########
                        real_current_series_583=pd.Series(data.iloc[583,27:58].values)
                        voltage_result_series_583=pd.Series(voltage_result_list_583[0,0:31])
                        corr_ann_583 = round(voltage_result_series_583.corr(real_current_series_583), 4)
                        rmse_val_583= rmse(data.iloc[583,27:58].values,voltage_result_list_583[0,0:31])
                        criteria_corr=0.95
                        criteria_rmse=0.05
                        print(corr_ann_235,corr_ann_456,corr_ann_381,corr_ann_619,corr_ann_582,corr_ann_583,rmse_val_235,rmse_val_456,rmse_val_381,rmse_val_619,rmse_val_582,rmse_val_583)                            
                        if corr_ann_235>criteria_corr and corr_ann_456>criteria_corr and corr_ann_381>criteria_corr and corr_ann_619>criteria_corr and corr_ann_582>criteria_corr and corr_ann_583>criteria_corr and rmse_val_235<criteria_rmse and rmse_val_456<criteria_rmse and rmse_val_381<criteria_rmse and rmse_val_619<criteria_rmse and rmse_val_582<criteria_rmse and rmse_val_583<criteria_rmse:
                            fig1 = plt.figure()
                            ax = fig1.add_subplot(111)
                            ax.scatter(x_current[0:21],voltage_result_list_381[0,0:21],color='red',label='Predicted Voltage ')
                            ax.plot(x_current[0:21],data.iloc[381,27:48].values,color='orange')
                            ax.set_xlabel(u"Current(mA cm^-2)")
                            ax.set_ylabel(u"Voltage(V)")
                            ax2 = ax.twinx()
                            ax2.set_ylabel(u'Power Density (mW cm^-2)')
                            ax2.scatter(x_current[0:21],predict_power_381[0:21],color='green',label='Predicted Power Density ')
                            ax2.plot(x_current[0:21],predict_power_real_381[0:21],color='blue')
                            plt.savefig('dbn_381.png')
                            print(voltage_result_list_381[0:21])
                            print(data.iloc[381,27:48].values)
                            print(predict_power_381[0:21])
                            print(predict_power_real_381[0:21])


                            fig2 = plt.figure()
                            ax = fig2.add_subplot(111)
                            ax.scatter(x_current[0:21],voltage_result_list_619[0,0:21],color='red',label='Predicted Voltage ')
                            ax.plot(x_current[0:21],data.iloc[619,27:48].values,color='orange')
                            ax.set_xlabel(u"Current(mA cm^-2)")
                            ax.set_ylabel(u"Voltage(V)")
                            ax2 = ax.twinx()
                            ax2.set_ylabel(u'Power Density (mW cm^-2)')
                            ax2.scatter(x_current[0:21],predict_power_619[0:21],color='green',label='Predicted Power Density ')
                            ax2.plot(x_current[0:21],predict_power_real_619[0:21],color='blue')
                            plt.savefig('dbn_619.png')
                            print(voltage_result_list_619[0:21])
                            print(data.iloc[619,27:48].values)
                            print(predict_power_619[0:21])
                            print(predict_power_real_619[0:21])


                            fig3 = plt.figure()
                            ax = fig3.add_subplot(111)
                            ax.scatter(x_current[0:21],voltage_result_list_235[0,0:21],color='red',label='Predicted Voltage ')
                            ax.plot(x_current[0:21],data.iloc[235,27:48].values,color='orange')
                            ax.set_xlabel(u"Current(mA cm^-2)")
                            ax.set_ylabel(u"Voltage(V)")
                            ax2 = ax.twinx()
                            ax2.set_ylabel(u'Power Density (mW cm^-2)')
                            ax2.scatter(x_current[0:21],predict_power_235[0:21],color='green',label='Predicted Power Density ')
                            ax2.plot(x_current[0:21],predict_power_real_235[0:21],color='blue')
                            plt.savefig('dbn_235.png')
                            print(voltage_result_list_235[0:21])
                            print(data.iloc[235,27:48].values)
                            print(predict_power_235[0:21])
                            print(predict_power_real_235[0:21])

                            fig4 = plt.figure()
                            ax = fig4.add_subplot(111)
                            ax.scatter(x_current[0:17],voltage_result_list_456[0,0:17],color='red',label='Predicted Voltage ')
                            ax.plot(x_current[0:17],data.iloc[456,27:44].values,color='orange')
                            ax.set_xlabel(u"Current(mA cm^-2)")
                            ax.set_ylabel(u"Voltage(V)")
                            ax2 = ax.twinx()
                            ax2.set_ylabel(u'Power Density (mW cm^-2)')
                            ax2.scatter(x_current[0:17],predict_power_456[0:17],color='green',label='Predicted Power Density ')
                            ax2.plot(x_current[0:17],predict_power_real_456[0:17],color='blue')
                            plt.savefig('dbn_456.png')
                            print(voltage_result_list_456[0:17])
                            print(data.iloc[456,27:44].values)
                            print(predict_power_456[0:17])
                            print(predict_power_real_456[0:17])

                            fig5 = plt.figure()
                            ax = fig5.add_subplot(111)
                            ax.scatter(x_current[0:26],voltage_result_list_582[0,0:26],color='red',label='Predicted Voltage ')
                            ax.plot(x_current[0:26],data.iloc[582,27:53].values,color='orange')
                            ax.set_xlabel(u"Current(mA cm^-2)")
                            ax.set_ylabel(u"Voltage(V)")
                            ax2 = ax.twinx()
                            ax2.set_ylabel(u'Power Density (mW cm^-2)')
                            ax2.scatter(x_current[0:26],predict_power_582[0:26],color='green',label='Predicted Power Density ')
                            ax2.plot(x_current[0:26],predict_power_real_582[0:26],color='blue')
                            plt.savefig('dbn_582.png')
                            print(voltage_result_list_582[0:26])
                            print(data.iloc[582,27:53].values)
                            print(predict_power_582[0:26])
                            print(predict_power_real_582[0:26])

                            fig6 = plt.figure()
                            ax = fig6.add_subplot(111)
                            ax.scatter(x_current[0:31],voltage_result_list_583[0,0:31],color='red',label='Predicted Voltage ')
                            ax.plot(x_current[0:31],data.iloc[583,27:58].values,color='orange')
                            ax.set_xlabel(u"Current(mA cm^-2)")
                            ax.set_ylabel(u"Voltage(V)")
                            ax2 = ax.twinx()
                            ax2.set_ylabel(u'Power Density (mW cm^-2)')
                            ax2.scatter(x_current[0:31],predict_power_583[0:31],color='green',label='Predicted Power Density ')
                            ax2.plot(x_current[0:31],predict_power_real_583[0:31],color='blue')
                            plt.savefig('dbn_583.png')
                            print(voltage_result_list_583[0:31])
                            print(data.iloc[583,27:58].values)
                            print(predict_power_583[0:31])
                            print(predict_power_real_583[0:31])

                            print(corr_ann_235,corr_ann_456,corr_ann_381,corr_ann_619,corr_ann_582,corr_ann_583,rmse_val_235,rmse_val_456,rmse_val_381,rmse_val_619,rmse_val_582,rmse_val_583)
                            print(neuron_num)
                            print(lr_rbm,lr,n_epoch,n_iter_back,bs)
                        else:continue
                        break
                    else:continue
                    break
                else:continue
                break
            else:continue
            break
        else:continue
        break
    else:continue
    break
print('finished')
                            

